In [1]:
# Import Libraries #
import findspark
findspark.find()
findspark.init()
from pyspark.sql.functions import split, col, array_contains, translate, round, size, when, udf, lit, mean, count, format_number, collect_list, expr
from pyspark.sql.types import TimestampType, MapType, IntegerType, StringType, ArrayType, FloatType, StructField, StructType
from pyspark.sql import SparkSession
from helper import *

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

# moves to FEN
konverter lista av moves til fen, at any given stage
eventuelt show board som 8x8 matrix


In [424]:
### sample moves list
import numpy as np

moves = ["'e4'", " 'c6'", " 'Nf3'", " 'd5'", " 'e5'", " 'e6'", " 'd4'", " 'Ne7'", " 'Bd3'", " 'c5'", " 'dxc5'",
 " 'Nd7'", " 'Be3'", " 'Qc7'", " 'b4'", " 'Nxe5'"," 'Nxe5'"," 'Qxe5'"," 'Bb5+'", " 'Bd7'"," 'Bxd7+'", " 'Kxd7'",
 " 'Qd4'", " 'Ng6'", " 'O-O'", " 'Qxd4'", " 'Bxd4'", " 'e5'", " 'Bb2'", " 'a5'", " 'b5'", " 'Bxc5'", " 'Nd2'",
 " 'b6'", " 'c4'", " 'd4'"," 'Ne4'", " 'Nf4'", " 'Nxc5+'", " 'bxc5'", " 'Ba3'", " 'Kd6'", " 'Rab1'"," 'Rhc8'",
 " 'g3'", " 'Nd3'", " 'Rfd1'", " 'Nb4'", " 'f4'", " 'e4'", " 'g4'", " 'e3'", " 'f5'", " 'Nxa2'", " 'Rf1'",
 " 'Nc3'", " 'Rbc1'", " 'e2'", " 'Rf4'", " 'Re8'", " 'Rxd4+'", " 'Ke5'", " 'Rd2'", " 'Kf6'", " 'Bb2'", " 'e1=Q+'",
 " 'Rxe1'", " 'Rxe1+'", " 'Kf2'", " 'Rae8'", " 'Bxc3+'", " 'Kg5'", " 'Bxa5'", " 'Rh1'", " 'Kg3'", " 'Re3+'", " 'Kg2'",
 " 'Rd1'", " 'Rxd1'"]


In [479]:
### helper funcs for getting position and piece, and undeveloped castle func
def piece_pos(move, piece_idx, pos_idx):
    pieces_list = [*"-prnbqk"]   # pieces
    hor_axis = [*"abcdefgh"] 
    piece =  pieces_list.index(move[piece_idx].lower())
    y,x =  hor_axis.index(move[pos_idx[0]]), int(move[pos_idx[1]])-1
    pos = [7-x, y]

    # pos = [7 - int(move[pos_idx[1]])-1, hor_axis.index(move[pos_idx[0]])]

    return piece, pos

def get_piece(move, piece_idx):
    pieces_list = [*"-prnbqk"]   # pieces
    piece = pieces_list.index(move[piece_idx].lower())
    return piece

def get_pos(move, pos_idx):
    hor_axis = [*"abcdefgh"] 
    # Ne4 -> N, 4, 4 -> N, 8-3, 3
    y,x =  hor_axis.index(move[pos_idx[0]]), int(move[pos_idx[1]])-1
    pos = [7-x, y]
    # print(f"org: {move[pos_idx[0]]} {int(move[pos_idx[1]])} , py indx: {hor_axis.index(move[pos_idx[0]])} {int(move[pos_idx[1]])-1}")
    # e4 -> 4e -> 4 4 -> 
    return pos

def castle(move):
    if len(move) == 3:
        #kingside
        print("kingside")
    else:
        #queenside
        print("queenside")

# char, int -> pawn move
# CHAR, char, int -> officer move
# char, x, char, int -> pawn take piece
# CHAR, x, char, int -> officeer take piece
# move, + -> check
# char, int, =, Q -> pawn becomes queen

In [464]:
### determines what pieces makes what move, checks for takes n stuff
def get_piece_pos(move): # variants: e4, Nd6, e4+, Nd6+ , bxc5 , Bxd4, O-O, O-O-O, e1=Q+
    if "+" in move:
        check = True
        move.remove(-1)
        print(f"Check")

    if "x" in move:             #take
        take = True
        split = move.index("x") 
        if "=" not in move:
            piece, pos = get_piece(move, piece_idx=split-1), get_pos(move, pos_idx=[split+1, split+2])
            print(f"Pawn {piece} takes {pos} and becomes Queen")
        else: #take piece for queening: dxe1=Q
            piece, pos = 5, get_pos(move, pos_idx=[split+1, split+2])
            print(f"{ ['', 'Pawn', 'Rook', 'Knight', 'Bishop', 'Queen', 'King'][abs(piece)]} takes {pos}")

    elif "-" in move:           #castle
        castle(move)
        

    elif "=" in move:           #queening
        piece, pos = 5, get_pos(move, [split+1,split+2])
        print(f"Pawn {piece} move to {pos} and becomes Queen")
    
    else:
        if len(move) == 2:
            piece, pos = 1, get_pos(move, pos_idx=[0,1])
            print(f"{ ['', 'Pawn', 'Rook', 'Knight', 'Bishop', 'Queen', 'King'][abs(piece)]} move to {pos}")
        elif len(move) == 3:
            piece, pos = piece_pos(move, piece_idx=0, pos_idx=[1,2])
            print(f"{ ['', 'Pawn', 'Rook', 'Knight', 'Bishop', 'Queen', 'King'][abs(piece)]} move to {pos}")

    return piece, pos
    

In [465]:
### checks and return the piece that makes the move
def get_valid_piece(pos, piece, pieces): # sett på ordet piece så mye nu at d ser feil ut
    # for what piece in the list of pieces is the new pos valid
    # rules: pawn move one forward, N two x one, R one axis the same, B same x as y, 
    # pieces_list = [*"-prnbqk"]
    if abs(piece) == 1:     # for pawn, either y increase by 1 or 2, or both x,y increace by 1
        for idx, elem in enumerate(pieces): 
            x_diff = (pos[0]-elem[0])
            y_diff = (pos[1]-elem[1])
            # print(f" pawn: x{x_diff}, y{y_diff}")
            if piece==1 and ((x_diff==-1 and y_diff==0) or (x_diff==-2 and y_diff==0)):
                valid_piece = idx
                # print(f"pos: {pos}, old: {pieces[idx]}")
                return valid_piece 
            elif piece==-1 and ((x_diff==1 and y_diff==0) or (x_diff==2 and y_diff==0)):
                valid_piece = idx
                # print(f"B: pos: {pos}, old: {pieces[idx]}")
                return valid_piece 

    if abs(piece) == 2: # For rook, where x or y coordinate stays the same.
        for idx, elem in enumerate(pieces): # foreach possible piece, one value must be the same as the new pos
            valid_piece = idx if elem[0]==pos[0] or elem[1]==pos[1] else None

    if abs(piece) == 3:     # for night, x increace 2 and y 1, or y 2 and x 1
        for idx, elem in enumerate(pieces): 
            x_diff = abs(pos[0]-elem[0]) 
            y_diff = abs(pos[1]-elem[1])

            if ((x_diff==2 and y_diff==1) or (x_diff==1 and y_diff==2)):
                valid_piece = idx
                # print(f"pos: {pos}, old: {pieces[idx]}")
                return valid_piece 


    if abs(piece) == 4:     # for bishop, where x and y increace the same
        for idx, elem in enumerate(pieces): # foreach possible piece, the x and y diff must be the same
            if abs(pos[0]-elem[0]) == abs(pos[1]-elem[1]):
                valid_piece = idx
                return valid_piece

    print(f"if you made it here, you fucked up somewhere.. \n pos: {pos} piece:{piece} , {pieces}")
    

In [466]:
### just for the validation, making a pairwise array of possible pieces
def format_pieces(pieces):
    formatted_pieces = []
    for idx in range(len(pieces[0])):
        formatted_pieces.append([pieces[0][idx],pieces[1][idx]])
    return formatted_pieces

In [473]:
### updating board, adding new values, removing old
def update_board(board, pos, piece):
    # set piece old pos = 0, and set piece new pos
    if abs(piece) == 4 or abs(piece) == 5:  # king and queen, only one
        print(f"K/Q old pos: {board[np.where(board==piece)]}")
        board[np.where(board==piece)] = 0
    else:
        # find what piece moved
        pieces = (np.where(board == piece)) #list of current pieces positions [[a,b]...[x,y]]
        pieces = format_pieces(pieces)
        #
        valid_piece = get_valid_piece(pos, piece, pieces)
        prev_pos = pieces[valid_piece]
        # print(f"prev_pos {prev_pos}")
        # set previous piece = 0
        # matrix[ned][bort]
        board[prev_pos[0], prev_pos[1]] = 0

    # update new pos. e mixup på x og y på disse as
    # x: 0,1,2,3,4,5,6,7 -> a,b,c,d,e,f,g,h
    # y: 0,1,2,3,4,5,6,7 -> 1,2,3,4,5,6,7,8
    board[pos[0], pos[1]] = piece

    return board

In [474]:
### makes the move, updates board and prints
def make_move(curr_board, col, move):
    mv_chars = [*move.strip(" ' '")]        # move string to list               

    piece, pos = get_piece_pos(mv_chars)             # determine what type of move
    # print(f"piece: {piece}")
    pieces_list = [*"-prnbqk"]   # pieces
    # print(f"color: {['white','black'][col==-1]}")
    # print(f"move: {move} piece: {pieces_list[piece]} new_pos: {pos} -den e motsatt av vanli")    

    piece = piece * col     # 1 .. 5 * 1 eller -1
    updated_board = update_board(curr_board, pos, piece) # update må finn piecen som har mova, sett den til 0 og sett ny pos til piece
    # print(updated_board)

    return updated_board

In [475]:
### Board to FEN and FEN to board, works ithink

def board_to_fen(board): #ex: matrix -> 'r1b1k1nr/p2p1pNp/n2B4/1p1NP2P/6P1/3P1Q2/P1P1K3/q5b1'
    # fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # start pos
    # enten update fen for hvert move, eller finn ut ka slutt posisjon e
    # kan lag en matrix
    # board_mat
    pieces_list = [*"0prnbqk"]   # pieces
    fen = ""
    for i in range(board.shape[0]): #number of rows
        zero_count = 0
        for j in board[i]:
            j = int(j)
            if j==0:
                zero_count += 1
            else:
                fen += str(zero_count) if zero_count>0 else ""
                zero_count = 0
                fen += pieces_list[abs(j)].upper() if j>0 else pieces_list[abs(j)]

        if j==board[i][-1] and zero_count != 0:
            fen += str(zero_count)
        
        fen += "/"
    
    return fen

# THIS SHIT WORKS FRFR
def fen_to_board(fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"):
    # w e pos, b e neg. p = 1, r=1, n=2, b=3, q=4, k=5
    board = np.zeros((8,8)) # empty game board
    fen = fen.split('/')        # line by line
    pieces_list = [*"prnbqk"]   # pieces
    number_list = [*'12345678']
    for idx, line in enumerate(fen):
        pieces = []
        for chr in line:
            if chr.lower() in pieces_list and chr.isupper():
                pieces.append(pieces_list.index(chr.lower())+1)
            elif  chr.lower() in pieces_list and chr.islower():
                pieces.append((pieces_list.index(chr.lower())+1)*-1)
            elif chr in number_list:
                for i in range(int(chr)):
                    pieces.append(0)
        print(pieces)
        board[idx] = np.array(pieces) 

    return board

def compare_fen(fen1, fen2):
    fen1 = fen1.split('/')
    fen2 = fen2.split('/')
    for idx, line in enumerate(fen1):
        if fen1[idx] != fen2[idx]:
            print(f"org: {fen1[idx]} - {fen2[idx]} new")



In [481]:
### making and updating the board with moves from the sample list
curr_board = fen_to_board()
# print(curr_board)

for idx, move in enumerate(moves):
    # matrix[ned][bort]
    col = 1 if idx % 2 == 0 else -1 # white or black turn
    print(f"turn: {idx+1}, move: {move} color: {['white','black'][col==-1]}")
    
    curr_board = make_move(curr_board, col, move)
    
end_fen = board_to_fen(curr_board)
end_fen

[-2, -3, -4, -5, -6, -4, -3, -2]
[-1, -1, -1, -1, -1, -1, -1, -1]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6, 4, 3, 2]
turn: 1, move: 'e4' color: white
Pawn move to [4, 4]
[[-2. -3. -4. -5. -6. -4. -3. -2.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.]
 [ 2.  3.  4.  5.  6.  4.  3.  2.]]
turn: 2, move:  'c6' color: black
Pawn move to [2, 2]
[[-2. -3. -4. -5. -6. -4. -3. -2.]
 [-1. -1.  0. -1. -1. -1. -1. -1.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  1.  1.  1.]
 [ 2.  3.  4.  5.  6.  4.  3.  2.]]
turn: 3, move:  'Nf3' color: white
Knight move to [5, 5]
[[-2. -3. -4. -5. -6. -4. -3. -2.]
 [-1. -1.  0. -1. -1. -

ValueError: 'd' is not in list

In [477]:
### testing the FEN convertion functions
# fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # clean board
fen = "rnbqKbnr/p4ppp/p1p1p1p1/RrRr2bB/8/2KKQQ2/PPPPPPPP/RNBQKBNR"
x = fen_to_board(fen)
new = board_to_fen(x)
print(new)

compare_fen(fen, new)

[-2, -3, -4, -5, 6, -4, -3, -2]
[-1, 0, 0, 0, 0, -1, -1, -1]
[-1, 0, -1, 0, -1, 0, -1, 0]
[2, -2, 2, -2, 0, 0, -4, 4]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 6, 6, 5, 5, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[2, 3, 4, 5, 6, 4, 3, 2]
rnbqKbnr/p4ppp/p1p1p1p1/RrRr2bB/8/2KKQQ2/PPPPPPPP/RNBQKBNR/


In [478]:
### shit code som kunne vært sletta, men tørr ikke incase magic logic somewhere
# [1,2,3][1==1]
# fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # start pos

# def make_board(): # w e pos, b e neg. p = 1, r=1, n=2, b=3, q=4, k=5
#     x = np.zeros((8,8)) # empty game board

#     fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR" # start pos
#     fen = fen.split('/')        # line by line
#     pieces_list = [*"prnbqk"]   # pieces
#     for idx, line in enumerate(fen):
#         if any(elem.lower() in pieces_list for elem in [*line]):    
#             pieces = [pieces_list.index(piece.lower())+1 for piece in [*line]]
#             pieces = [piece * -1 if line.islower() else piece for piece in pieces] #makin black
#             x[idx] = np.array(pieces)

#     return x

# import numpy as np
# def fen_to_board2(fen):
#     # w e pos, b e neg. p = 1, r=1, n=2, b=3, q=4, k=5
#     board = np.zeros((8,8)) # empty game board

    
#     fen = fen.split('/')        # line by line
#     pieces_list = [*"prnbqk"]   # pieces
#     number_list = [*'12345678']
#     for idx, line in enumerate(fen):
#         if any(elem.lower() in pieces_list for elem in [*line]):    
#             pieces = [pieces_list.index(piece.lower())+1 for piece in [*line]]
#             pieces = [piece * -1 if line.islower() else piece for piece in pieces] #makin black
#             board[idx] = np.array(pieces)
#         elif any(elem in number_list for elem in [*line]):

#     return board

# board = fen_to_board(fen)
# print(board)
# test = np.zeros((8,8))
# for i in range(8):
#     test[i][i] = i

# test[4,3] = 10
# test
#     if mv == 2: 
#         piece = 1   
#         pos = [hor_axis.index(move[0]), int(move[1])]
#         print(f"{move} bonde")
#     elif mv == 3:
#         if "+" in move: # e4+
#             piece = 1
#             pos = [hor_axis.index(move[0]), int(move[1])]

#         elif "-" in move:   # O - O 
#             print("castle")
#         else:       # stnd officer move: Ne4
#             piece = pieces_list.index(move[0].lower())
#             pos = [hor_axis.index(move[1]), int(move[2])]
#             print(f"{move} {pieces_list[piece]} officer")

#     elif mv == 4:
#         if "x" in move:
#             piece = pieces_list.index(move[0].lower()) if move[0].isupper() else 1
#             split = move.index("x")     # Nxe4 -> N x e4
#             pos = [hor_axis.index(move[split + 1]), int(move[split + 2])]

#             print(f"{move} take")
#     else: 
#         if "+" in move:
#             print(f"{move} check")
#         elif "=" in move:
#             print(f"{move} pawn to queen")
#         elif "x" in move:
#             print(f"take! {move}")
#             move = move.split('x')  # piece x pos
#             piece = pieces_list.index(move[0].lower()) #dxe1=Q+

    
# def get_new_pos(move):                  # move in list form
#     hor_axis = [*"abcdefgh"]            # letter to number
#     if "="  in move:   # pawn to queen. 
#         new_pos = [move[0], move[1]]  # take the position arguments
#         new_pos = [hor_axis.index(new_pos[0]), int(new_pos[1])] # converte them to numbers

#     elif "-" in move: # castle. both king and rook new pos
#         pass
    
#     elif "+" not in move:                 # if the move is a check, the '+' is at the end of the list
#         new_pos = [move[-2], move[-1]]  # take the position arguments
#         new_pos = [hor_axis.index(new_pos[0]), int(new_pos[1])] # converte them to numbers
    
#     else:                               # if the move is a check, then
#         new_pos = [move[-3], move[-2]]
#         new_pos = [hor_axis.index(new_pos[0]), int(new_pos[1])]

#     return new_pos